In [1]:
#@title Imports
import pandas as pd
import io
import numpy as np
import matplotlib.pyplot as plt
import re

In [2]:
#@title Activating the GPU
# Main menu->Runtime->Change Runtime Type
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [3]:
#@title Further settings to be done
!pip install transformers
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
import torch
import transformers as ppb
import warnings
warnings.filterwarnings('ignore')

     |████████████████████████████████| 2.0MB 9.3MB/s 
     |████████████████████████████████| 3.2MB 47.9MB/s 
     |████████████████████████████████| 890kB 51.0MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=7ccdeee1c5d8fd1ebb90ba9deba66f283d853fdd3230d49613595be7dab4829c
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [4]:
#@title Specifying CUDA as the device for Torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
torch.cuda.get_device_name(0)

'Tesla T4'

In [6]:
#@title Import dataset
df = pd.read_csv("SentiFM.csv")

In [8]:
df.head()

,Unnamed: 0,indexes,text,label
0,0,5750.0,It will not say what it has spent on the proje...,-1
1,1,4305.0,"Sir John Bond , chairman , told the bank 's an...",-1
2,2,7684.0,Unilever was criticised by shareholders at its...,-1
3,3,5360.0,Mr Sarin 's statement came as he announced tha...,-1
4,4,1887.0,BASF was advised by Deutsche Bank .,-1


In [9]:
df['label'].unique().tolist()

['-1',
 'ShareRepurchase',
 'Profit',
 'QuarterlyResultsProfit',
 'Dividend',
 'QuarterlyResults',
 'BuyRating',
 'SalesVolume',
 'QuarterlyResultsSalesVolume',
 'Turnover',
 'MergerAcquisition',
 'ProfitMergerAcquisition',
 'MergerAcquisitionTurnoverSalesVolume',
 'ProfitSalesVolume',
 'TargetPrice',
 'ProfitShareRepurchase',
 'Debt',
 'QuarterlyResultsTurnover',
 'TargetPriceBuyRating',
 'ProfitTurnover',
 'MergerAcquisitionSalesVolume',
 'MergerAcquisitionDebt',
 'DividendProfit',
 'TurnoverSalesVolume',
 'DebtShareRepurchase',
 'DividendShareRepurchase',
 'DividendTurnover',
 'QuarterlyResultsMergerAcquisition',
 'ProfitBuyRating',
 'QuarterlyResultsProfitSalesVolume',
 'QuarterlyResultsTurnoverSalesVolume',
 'ProfitTurnoverSalesVolume',
 'DividendQuarterlyResultsProfit',
 'DividendProfitSalesVolume',
 'SalesVolumeBuyRating',
 'DividendDebt',
 'DividendMergerAcquisition',
 'QuarterlyResultsProfitTurnover',
 'MergerAcquisitionBuyRating',
 'DividendMergerAcquisitionShareRepurchase',


In [10]:
#@title Mutate targets
profit = df.label.values
profit = [float(value.find('Profit')>=0) for value in profit]
profit_df = pd.DataFrame(
    {'Index': df["indexes"].values,
     'Text':df["text"].values,
     'Label':profit
     }
)

QuarterlyResults = df.label.values
QuarterlyResults = [float(value.find('QuarterlyResults')>=0) for value in QuarterlyResults]
QuarterlyResults_df = pd.DataFrame(
    {'Index': df["indexes"].values,
     'Text':df["text"].values,
     'Label':QuarterlyResults
     }
)

BuyRating = df.label.values
BuyRating = [float(value.find('BuyRating')>=0) for value in BuyRating]
BuyRating_df = pd.DataFrame(
    {'Index': df["indexes"].values,
     'Text':df["text"].values,
     'Label':BuyRating
     }
)

Debt = df.label.values
Debt = [float(value.find('Debt')>=0) for value in Debt]
Debt_df = pd.DataFrame(
    {'Index': df["indexes"].values,
     'Text':df["text"].values,
     'Label':Debt
     }
)

Dividend = df.label.values
Dividend = [float(value.find('Dividend')>=0) for value in Dividend]
Dividend_df = pd.DataFrame(
    {'Index': df["indexes"].values,
     'Text':df["text"].values,
     'Label':Dividend
     }
)


SalesVolume = df.label.values
SalesVolume = [float(value.find('SalesVolume')>=0) for value in SalesVolume]
SalesVolume_df = pd.DataFrame(
    {'Index': df["indexes"].values,
     'Text':df["text"].values,
     'Label':SalesVolume
     }
)


MergerAcquisition = df.label.values
MergerAcquisition = [float(value.find('MergerAcquisition')>=0) for value in MergerAcquisition]
MergerAcquisition_df = pd.DataFrame(
    {'Index': df["indexes"].values,
     'Text':df["text"].values,
     'Label':MergerAcquisition
     }
)

Turnover = df.label.values
Turnover = [float(value.find('Turnover')>=0) for value in Turnover]
Turnover_df = pd.DataFrame(
    {'Index': df["indexes"].values,
     'Text':df["text"].values,
     'Label':Turnover
     }
)

TargetPrice = df.label.values
TargetPrice = [float(value.find('TargetPrice')>=0) for value in TargetPrice]
TargetPrice_df = pd.DataFrame(
    {'Index': df["indexes"].values,
     'Text':df["text"].values,
     'Label':TargetPrice
     }
)

ShareRepurchase = df.label.values
ShareRepurchase = [float(value.find('ShareRepurchase')>=0) for value in ShareRepurchase]
ShareRepurchase_df = pd.DataFrame(
    {'Index': df["indexes"].values,
     'Text':df["text"].values,
     'Label':ShareRepurchase
     }
)

In [11]:
print(
set(profit),
set(QuarterlyResults),
set(BuyRating),
set(Debt),
set(Dividend),
set(SalesVolume),
set(MergerAcquisition),
set(Turnover),
set(TargetPrice),
set(ShareRepurchase))

{0.0, 1.0} {0.0, 1.0} {0.0, 1.0} {0.0, 1.0} {0.0, 1.0} {0.0, 1.0} {0.0, 1.0} {0.0, 1.0} {0.0, 1.0} {0.0, 1.0}


In [12]:
#@title Import BERT from HuggingFace
# For DistilBERT:
model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')

## Want BERT instead of distilBERT? Uncomment the following line:
#model_class, tokenizer_class, pretrained_weights = (ppb.BertModel, ppb.BertTokenizer, 'bert-base-uncased')

# Load pretrained model/tokenizer
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

In [15]:
#@title Preparing the dataset
profit_df_batch_1 = profit_df[:2000]
profit_df_batch_1['Label'].value_counts()

0.0    1849
1.0     151
Name: Label, dtype: int64

In [16]:
QuarterlyResults_df_batch_1 = QuarterlyResults_df[:2000]
QuarterlyResults_df_batch_1['Label'].value_counts()

0.0    1943
1.0      57
Name: Label, dtype: int64

In [17]:
BuyRating_df_batch_1 = BuyRating_df[:2000]
BuyRating_df_batch_1['Label'].value_counts()

0.0    1945
1.0      55
Name: Label, dtype: int64

In [18]:
Debt_df_batch_1 = Debt_df[:2000]
Debt_df_batch_1['Label'].value_counts()


0.0    1988
1.0      12
Name: Label, dtype: int64

In [19]:
Dividend_df_batch_1 = Dividend_df[:2000]
Dividend_df_batch_1['Label'].value_counts()

0.0    1962
1.0      38
Name: Label, dtype: int64

In [20]:
SalesVolume_df_batch_1 = SalesVolume_df[:2000]
SalesVolume_df_batch_1['Label'].value_counts()

0.0    1914
1.0      86
Name: Label, dtype: int64

In [21]:
Turnover_df_batch_1 = Turnover_df[:2000]
Turnover_df_batch_1['Label'].value_counts()

0.0    1951
1.0      49
Name: Label, dtype: int64

In [22]:
MergerAcquisition_df_batch_1 = MergerAcquisition_df[:2000]
MergerAcquisition_df_batch_1['Label'].value_counts()

0.0    1951
1.0      49
Name: Label, dtype: int64

In [23]:
TargetPrice_df_batch_1 = TargetPrice_df[:2000]
TargetPrice_df_batch_1['Label'].value_counts()

0.0    1982
1.0      18
Name: Label, dtype: int64

In [24]:
ShareRepurchase_df_batch_1 = ShareRepurchase_df[:2000]
ShareRepurchase_df_batch_1['Label'].value_counts()

0.0    1988
1.0      12
Name: Label, dtype: int64

In [21]:
tokenized_profit = profit_df_batch_1['Text'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))

In [22]:
tokenized_profit

0       [101, 2009, 2097, 2025, 2360, 2054, 2009, 2038...
1       [101, 2909, 2198, 5416, 1010, 3472, 1010, 2409...
2       [101, 4895, 9463, 6299, 2001, 10648, 2011, 153...
3       [101, 2720, 18906, 2378, 1005, 1055, 4861, 223...
4       [101, 19021, 2546, 2001, 9449, 2011, 11605, 29...
                              ...                        
1995    [101, 11605, 2924, 2056, 1996, 4753, 2001, 100...
1996    [101, 2414, 4518, 3863, 1010, 2125, 1014, 1012...
1997    [101, 1996, 2177, 2988, 2561, 4341, 2039, 1021...
1998    [101, 1000, 2057, 2079, 2025, 2156, 1037, 6918...
1999    [101, 1996, 2801, 2001, 2000, 24647, 6304, 202...
Name: Text, Length: 2000, dtype: object

In [23]:
max_len = 0
for i in tokenized_profit.values:
    if len(i) > max_len:
        max_len = len(i)
padded_profit = np.array([i + [0]*(max_len-len(i)) for i in tokenized_profit.values])

In [24]:
padded_profit

array([[ 101, 2009, 2097, ...,    0,    0,    0],
       [ 101, 2909, 2198, ...,    0,    0,    0],
       [ 101, 4895, 9463, ...,    0,    0,    0],
       ...,
       [ 101, 1996, 2177, ...,    0,    0,    0],
       [ 101, 1000, 2057, ...,    0,    0,    0],
       [ 101, 1996, 2801, ...,    0,    0,    0]])

In [25]:
np.array(padded_profit).shape

(2000, 80)

In [26]:
#@title Masking padded elements 
attention_mask = np.where(padded_profit != 0, 1, 0)
attention_mask.shape

(2000, 80)

In [27]:
input_ids = torch.tensor(padded_profit)  
attention_mask = torch.tensor(attention_mask)

with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)

In [28]:
features = last_hidden_states[0][:,0,:].numpy()

In [29]:
features.shape

(2000, 768)

In [30]:
labels = profit_df_batch_1['Label']

In [31]:
train_features, test_features, train_labels, test_labels = train_test_split(features, labels)

In [32]:
lr_clf = LogisticRegression()
lr_clf.fit(train_features, train_labels)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [33]:
lr_clf.score(test_features, test_labels)

0.936

In [34]:
from sklearn.dummy import DummyClassifier
clf = DummyClassifier()

scores = cross_val_score(clf, train_features, train_labels)
print("Dummy classifier score: %0.3f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Dummy classifier score: 0.867 (+/- 0.02)


In [35]:
profit_df_batch_1 = profit_df[:2000]
profit_df_batch_1['Label'].value_counts()

0.0    1849
1.0     151
Name: Label, dtype: int64

In [36]:
profit_df['Label'].value_counts()

0.0    9286
1.0     651
Name: Label, dtype: int64

In [14]:
tokenized_QuarterlyResults = QuarterlyResults_df_batch_1['Text'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))
max_len = 0
for i in tokenized_QuarterlyResults.values:
    if len(i) > max_len:
        max_len = len(i)
padded_QuarterlyResults= np.array([i + [0]*(max_len-len(i)) for i in tokenized_QuarterlyResults.values])
#@title Masking padded elements 
attention_mask = np.where(padded_QuarterlyResults != 0, 1, 0)
attention_mask.shape
input_ids = torch.tensor(padded_QuarterlyResults)  
attention_mask = torch.tensor(attention_mask)

with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)
features = last_hidden_states[0][:,0,:].numpy()
labels = QuarterlyResults_df_batch_1['Label']
train_features, test_features, train_labels, test_labels = train_test_split(features, labels)
lr_clf = LogisticRegression()
lr_clf.fit(train_features, train_labels)
print(lr_clf.score(test_features, test_labels))


NameError: ignored

In [38]:
from sklearn.dummy import DummyClassifier
clf = DummyClassifier()

scores = cross_val_score(clf, train_features, train_labels)
print("Dummy classifier score: %0.3f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Dummy classifier score: 0.946 (+/- 0.01)


In [39]:
tokenized_BuyRating = BuyRating_df_batch_1['Text'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))
max_len = 0
for i in tokenized_BuyRating.values:
    if len(i) > max_len:
        max_len = len(i)
padded_BuyRating= np.array([i + [0]*(max_len-len(i)) for i in tokenized_BuyRating.values])
#@title Masking padded elements 
attention_mask = np.where(padded_BuyRating != 0, 1, 0)
attention_mask.shape
input_ids = torch.tensor(padded_BuyRating)  
attention_mask = torch.tensor(attention_mask)

with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)
features = last_hidden_states[0][:,0,:].numpy()
labels = BuyRating_df_batch_1['Label']
train_features, test_features, train_labels, test_labels = train_test_split(features, labels)
lr_clf = LogisticRegression()
lr_clf.fit(train_features, train_labels)
print(lr_clf.score(test_features, test_labels))

0.976


In [40]:
from sklearn.dummy import DummyClassifier
clf = DummyClassifier()

scores = cross_val_score(clf, train_features, train_labels)
print("Dummy classifier score: %0.3f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Dummy classifier score: 0.950 (+/- 0.02)


In [41]:
tokenized_Debt = Debt_df_batch_1['Text'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))
max_len = 0
for i in tokenized_Debt.values:
    if len(i) > max_len:
        max_len = len(i)
padded_Debt= np.array([i + [0]*(max_len-len(i)) for i in tokenized_Debt.values])
#@title Masking padded elements 
attention_mask = np.where(padded_Debt != 0, 1, 0)
attention_mask.shape
input_ids = torch.tensor(padded_Debt)  
attention_mask = torch.tensor(attention_mask)

with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)
features = last_hidden_states[0][:,0,:].numpy()
labels = Debt_df_batch_1['Label']
train_features, test_features, train_labels, test_labels = train_test_split(features, labels)
lr_clf = LogisticRegression()
lr_clf.fit(train_features, train_labels)
print(lr_clf.score(test_features, test_labels))

0.992


In [42]:
from sklearn.dummy import DummyClassifier
clf = DummyClassifier()

scores = cross_val_score(clf, train_features, train_labels)
print("Dummy classifier score: %0.3f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Dummy classifier score: 0.990 (+/- 0.01)


In [43]:
tokenized_Dividend = Dividend_df_batch_1['Text'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))
max_len = 0
for i in tokenized_Dividend.values:
    if len(i) > max_len:
        max_len = len(i)
padded_Dividend= np.array([i + [0]*(max_len-len(i)) for i in tokenized_Dividend.values])
#@title Masking padded elements 
attention_mask = np.where(padded_Dividend != 0, 1, 0)
attention_mask.shape
input_ids = torch.tensor(padded_Dividend)  
attention_mask = torch.tensor(attention_mask)

with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)
features = last_hidden_states[0][:,0,:].numpy()
labels = Dividend_df_batch_1['Label']
train_features, test_features, train_labels, test_labels = train_test_split(features, labels)
lr_clf = LogisticRegression()
lr_clf.fit(train_features, train_labels)
print(lr_clf.score(test_features, test_labels))

0.978


In [44]:
from sklearn.dummy import DummyClassifier
clf = DummyClassifier()

scores = cross_val_score(clf, train_features, train_labels)
print("Dummy classifier score: %0.3f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Dummy classifier score: 0.969 (+/- 0.01)


In [45]:
tokenized_SalesVolume = SalesVolume_df_batch_1['Text'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))
max_len = 0
for i in tokenized_SalesVolume.values:
    if len(i) > max_len:
        max_len = len(i)
padded_SalesVolume= np.array([i + [0]*(max_len-len(i)) for i in tokenized_SalesVolume.values])
#@title Masking padded elements 
attention_mask = np.where(padded_SalesVolume != 0, 1, 0)
attention_mask.shape
input_ids = torch.tensor(padded_SalesVolume)  
attention_mask = torch.tensor(attention_mask)

with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)
features = last_hidden_states[0][:,0,:].numpy()
labels = SalesVolume_df_batch_1['Label']
train_features, test_features, train_labels, test_labels = train_test_split(features, labels)
lr_clf = LogisticRegression()
lr_clf.fit(train_features, train_labels)
print(lr_clf.score(test_features, test_labels))

0.97


In [46]:
from sklearn.dummy import DummyClassifier
clf = DummyClassifier()

scores = cross_val_score(clf, train_features, train_labels)
print("Dummy classifier score: %0.3f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Dummy classifier score: 0.910 (+/- 0.03)


In [47]:
tokenized_Turnover = Turnover_df_batch_1['Text'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))
max_len = 0
for i in tokenized_Turnover.values:
    if len(i) > max_len:
        max_len = len(i)
padded_Turnover= np.array([i + [0]*(max_len-len(i)) for i in tokenized_Turnover.values])
#@title Masking padded elements 
attention_mask = np.where(padded_Turnover != 0, 1, 0)
attention_mask.shape
input_ids = torch.tensor(padded_Turnover)  
attention_mask = torch.tensor(attention_mask)

with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)
features = last_hidden_states[0][:,0,:].numpy()
labels = Turnover_df_batch_1['Label']
train_features, test_features, train_labels, test_labels = train_test_split(features, labels)
lr_clf = LogisticRegression()
lr_clf.fit(train_features, train_labels)
print(lr_clf.score(test_features, test_labels))

0.97


In [48]:
from sklearn.dummy import DummyClassifier
clf = DummyClassifier()

scores = cross_val_score(clf, train_features, train_labels)
print("Dummy classifier score: %0.3f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Dummy classifier score: 0.949 (+/- 0.01)


In [49]:
tokenized_MergerAcquisition = MergerAcquisition_df_batch_1['Text'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))
max_len = 0
for i in tokenized_MergerAcquisition.values:
    if len(i) > max_len:
        max_len = len(i)
padded_MergerAcquisition= np.array([i + [0]*(max_len-len(i)) for i in tokenized_MergerAcquisition.values])
#@title Masking padded elements 
attention_mask = np.where(padded_MergerAcquisition != 0, 1, 0)
attention_mask.shape
input_ids = torch.tensor(padded_MergerAcquisition)  
attention_mask = torch.tensor(attention_mask)

with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)
features = last_hidden_states[0][:,0,:].numpy()
labels = MergerAcquisition_df_batch_1['Label']
train_features, test_features, train_labels, test_labels = train_test_split(features, labels)
lr_clf = LogisticRegression()
lr_clf.fit(train_features, train_labels)
print(lr_clf.score(test_features, test_labels))

0.972


In [50]:
from sklearn.dummy import DummyClassifier
clf = DummyClassifier()

scores = cross_val_score(clf, train_features, train_labels)
print("Dummy classifier score: %0.3f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Dummy classifier score: 0.961 (+/- 0.02)


In [51]:
tokenized_TargetPrice = TargetPrice_df_batch_1['Text'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))
max_len = 0
for i in tokenized_TargetPrice.values:
    if len(i) > max_len:
        max_len = len(i)
padded_TargetPrice= np.array([i + [0]*(max_len-len(i)) for i in tokenized_TargetPrice.values])
#@title Masking padded elements 
attention_mask = np.where(padded_TargetPrice != 0, 1, 0)
attention_mask.shape
input_ids = torch.tensor(padded_TargetPrice)  
attention_mask = torch.tensor(attention_mask)

with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)
features = last_hidden_states[0][:,0,:].numpy()
labels = TargetPrice_df_batch_1['Label']
train_features, test_features, train_labels, test_labels = train_test_split(features, labels)
lr_clf = LogisticRegression()
lr_clf.fit(train_features, train_labels)
print(lr_clf.score(test_features, test_labels))

0.992


In [52]:
from sklearn.dummy import DummyClassifier
clf = DummyClassifier()

scores = cross_val_score(clf, train_features, train_labels)
print("Dummy classifier score: %0.3f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Dummy classifier score: 0.983 (+/- 0.00)


In [11]:
tokenized_ShareRepurchase = ShareRepurchase_df_batch_1['Text'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))
max_len = 0
for i in tokenized_ShareRepurchase.values:
    if len(i) > max_len:
        max_len = len(i)
padded_ShareRepurchase= np.array([i + [0]*(max_len-len(i)) for i in tokenized_ShareRepurchase.values])
#@title Masking padded elements 
attention_mask = np.where(padded_ShareRepurchase != 0, 1, 0)
attention_mask.shape
input_ids = torch.tensor(padded_ShareRepurchase)  
attention_mask = torch.tensor(attention_mask)

with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)
features = last_hidden_states[0][:,0,:].numpy()
labels = ShareRepurchase_df_batch_1['Label']
train_features, test_features, train_labels, test_labels = train_test_split(features, labels)
lr_clf = LogisticRegression()
lr_clf.fit(train_features, train_labels)
print(lr_clf.score(test_features, test_labels))

0.998


In [54]:
from sklearn.dummy import DummyClassifier
clf = DummyClassifier()

scores = cross_val_score(clf, train_features, train_labels)
print("Dummy classifier score: %0.3f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Dummy classifier score: 0.989 (+/- 0.01)


In [25]:
def BERTembedSentence(data, tokenizer):
  tokenized = data['Text'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))
  max_len = 0
  for i in tokenized.values:
    if len(i) > max_len:
      max_len = len(i)
  padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])
  attention_mask = np.where(padded != 0, 1, 0)
  input_ids = torch.tensor(padded)  
  attention_mask = torch.tensor(attention_mask)
  with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)
  features = last_hidden_states[0][:,0,:].numpy()
  labels = data['Label']
  return(features, labels)

In [26]:
features, labels = BERTembedSentence(ShareRepurchase_df_batch_1, tokenizer)
labels
features

array([[-0.12621328, -0.23826519,  0.26356655, ...,  0.18527333,
         0.23833135,  0.43361786],
       [-0.18598606, -0.1330147 ,  0.02525789, ...,  0.01531765,
         0.18774836,  0.40838298],
       [-0.02733176, -0.12236849, -0.07800321, ..., -0.06941407,
         0.3088101 ,  0.6904435 ],
       ...,
       [-0.16239811, -0.19700176,  0.2515366 , ..., -0.04752159,
         0.42372584,  0.20651266],
       [ 0.0524352 , -0.14646558, -0.06007141, ...,  0.00241058,
         0.29696476,  0.25563154],
       [ 0.06571016, -0.13380173, -0.018204  , ..., -0.07853746,
         0.20437789,  0.33497295]], dtype=float32)

In [32]:
train_features, test_features, train_labels, test_labels = train_test_split(features, labels)
lr_clf = LogisticRegression()
lr_clf.fit(train_features, train_labels)
print(lr_clf.score(test_features, test_labels))

0.998


In [33]:
test_features

array([[ 0.03419941, -0.09250285,  0.09647837, ..., -0.08793031,
         0.26457712,  0.1201782 ],
       [-0.4313788 , -0.21317062, -0.01011679, ..., -0.05483741,
         0.612322  ,  0.22862172],
       [-0.18782383, -0.1588561 ,  0.11088724, ..., -0.12492155,
         0.27815625,  0.42056513],
       ...,
       [-0.27599877, -0.36201295,  0.19575432, ..., -0.11181227,
         0.5096465 ,  0.28587788],
       [-0.31041428, -0.05389386, -0.34992507, ..., -0.1814967 ,
         0.48965526, -0.04328017],
       [-0.12621328, -0.23826519,  0.26356655, ...,  0.18527333,
         0.23833135,  0.43361786]], dtype=float32)